# Conversation Buffer Memory

## Colab-specific setup

Make sure you have a database available. This is a Colab notebook, so here it is assumed you will use
an Astra DB cloud instance. For usage with an Apache Cassandra® cluster, switch to a local setup
as instructed [on cassio.org](https://cassio.org/more_info/#use-a-local-vector-capable-cassandra).

Get ready to supply the connection parameters: Database ID and Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details.
Remember you need a Token with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

Likewise, ensure you have the necessary secret(s) for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).

_Note: this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/langchain/memory-conversationbuffermemory/)._


In [ ]:
# install required dependencies
! pip install -q --progress-bar off \
    "git+https://github.com/hemidactylus/langchain@SL-preview-for-cassio#egg=langchain-core&subdirectory=libs/core" \
    "git+https://github.com/hemidactylus/langchain@SL-preview-for-cassio#egg=langchain&subdirectory=libs/langchain" \
    "cassio>=0.1.3" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2" 
exit()

⚠️ **Do not mind a "Your session crashed..." message you may see.**

It was us, making sure your kernel restarts with all the correct dependency versions. _You can now proceed with the notebook._

### DB connection parameters
Example values (found on your Astra dashboard):
- Database ID: `01234567-89ab-cdef-0123-456789abcdef`
- Token: `AstraCS:6gBhNmsk135....` (ensure it has a role of at least "Database Administrator")
- _Keyspace. Optional, if provided:_ `cassio_tutorials`

In [ ]:
import os
from getpass import getpass

# Input your Astra DB connection parameters and secrets:

os.environ["ASTRA_DB_ID"] = input("ASTRA_DB_ID = ")

os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

_keyspace = input("(Optional) ASTRA_DB_KEYSPACE = ")
if _keyspace:
    os.environ["ASTRA_DB_KEYSPACE"] = _keyspace

### LLM Provider

In the cell below you can choose between **GCP Vertex AI** or **OpenAI** for your LLM services.
(See [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for more details).

Make sure you set the `llmProvider` variable and supply the corresponding access secrets in the following cell.

In [ ]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI', 'Azure_OpenAI'


In [ ]:
from getpass import getpass
if llmProvider == 'OpenAI':
    apiSecret = getpass(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
elif llmProvider == 'Azure_OpenAI':
    # a few parameters must be input
    apiSecret = input(f'Your API Key for LLM provider "{llmProvider}": ')
    os.environ['AZURE_OPENAI_API_KEY'] = apiSecret
    apiBase = input('The "Base URL" for your models (e.g. "https://YOUR-RESOURCE-NAME.openai.azure.com"): ')
    os.environ['AZURE_OPENAI_API_BASE'] = apiBase
    apiLLMDepl = input('The name of your LLM Deployment: ')
    os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'] = apiLLMDepl
    apiLLMModel = input('The name of your LLM Model (e.g. "gpt-4"): ')
    os.environ['AZURE_OPENAI_LLM_MODEL'] = apiLLMModel
    apiEmbDepl = input('The name for your Embeddings Deployment: ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'] = apiEmbDepl
    apiEmbModel = input('The name of your Embedding Model (e.g. "text-embedding-ada-002"): ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'] = apiEmbModel

    # The following is probably not going to change for some time...
    os.environ['AZURE_OPENAI_API_VERSION'] = '2023-03-15-preview'
else:
    raise ValueError('Unknown/unsupported LLM Provider')

### Colab preamble completed

The following cells constitute the demo notebook proper.

# Conversation Buffer Memory

The "base memory class" seen in the previous example is now put to use in a higher-level abstraction provided by LangChain: 

In [1]:
from langchain.memory import CassandraChatMessageHistory
from langchain.memory import ConversationBufferMemory

A database connection is needed. _(If on a Colab, the only supported option is the cloud service Astra DB.)_

In [2]:
# Getting ready to initialize the DB connection globally ...
import os

import cassio

Select your choice of database by editing this cell, if needed:

In [3]:
database_mode = "astra_db"  # only "astra_db" supported on Colab

In [4]:
if database_mode == "astra_db":
    cassio.init(
        database_id=os.environ["ASTRA_DB_ID"],
        token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
        keyspace=os.environ.get("ASTRA_DB_KEYSPACE"),  # this is optional
    )

In [5]:
if database_mode == "cassandra":
    # Cassandra not supported on Colab - please define your own getCassandraCQLSession/getCassandraCQLKeyspace
    cassio.init(
        session=getCassandraCQLSession(),
        keyspace=getCassandraCQLKeyspace(),
    )

In [6]:
message_history = CassandraChatMessageHistory(
    session_id='conversation-0123',
    session=None,
    keyspace=None,
    ttl_seconds = 3600,
)
message_history.clear()

## Use in a ConversationChain

### Create a Memory

The Cassandra message history is specified:

In [7]:
cassBuffMemory = ConversationBufferMemory(
    chat_memory=message_history,
)

### Language model

Below is the logic to instantiate the LLM of choice. We chose to leave it in the notebooks for clarity.

In [8]:
import os
# creation of the LLM resources


if llmProvider == 'GCP_VertexAI':
    from langchain.llms import VertexAI
    llm = VertexAI()
    print('LLM from Vertex AI')
elif llmProvider == 'OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'open_ai'
    from langchain.llms import OpenAI
    llm = OpenAI()
    print('LLM from OpenAI')
elif llmProvider == 'Azure_OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'azure'
    os.environ['OPENAI_API_VERSION'] = os.environ['AZURE_OPENAI_API_VERSION']
    os.environ['OPENAI_API_BASE'] = os.environ['AZURE_OPENAI_API_BASE']
    os.environ['OPENAI_API_KEY'] = os.environ['AZURE_OPENAI_API_KEY']
    from langchain.llms import AzureOpenAI
    llm = AzureOpenAI(temperature=0, model_name=os.environ['AZURE_OPENAI_LLM_MODEL'],
                      engine=os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'])
    print('LLM from Azure OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM from OpenAI


### Create a chain

As the conversation proceeds, a growing history of past exchanges finds it way automatically to the prompt that the LLM receives:

In [9]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=cassBuffMemory,
)

In [10]:
conversation.predict(input="Hello, how can I roast an apple?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, how can I roast an apple?
AI:

> Finished chain.


' Hi there! Roasting an apple is a great way to bring out the flavor of the fruit. To roast an apple, preheat your oven to 375 degrees Fahrenheit. Cut the apple into thin slices, and spread them on a greased baking sheet. Sprinkle the apple slices with cinnamon, nutmeg, and brown sugar. Bake the apples in the oven for about 25 minutes, or until the edges are golden brown. Enjoy!'

In [11]:
conversation.predict(input="Can I do it on a bonfire?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, how can I roast an apple?
AI:  Hi there! Roasting an apple is a great way to bring out the flavor of the fruit. To roast an apple, preheat your oven to 375 degrees Fahrenheit. Cut the apple into thin slices, and spread them on a greased baking sheet. Sprinkle the apple slices with cinnamon, nutmeg, and brown sugar. Bake the apples in the oven for about 25 minutes, or until the edges are golden brown. Enjoy!
Human: Can I do it on a bonfire?
AI:

> Finished chain.


" Yes, you can definitely roast an apple over a bonfire! Start by spearing the apple onto a roasting stick. Hold the stick a few inches above the fire, and turn the apple every few minutes until it's evenly cooked. Once the apple looks and smells done, you can take it off the fire and enjoy!"

In [12]:
conversation.predict(input="What about a microwave, would the apple taste good?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, how can I roast an apple?
AI:  Hi there! Roasting an apple is a great way to bring out the flavor of the fruit. To roast an apple, preheat your oven to 375 degrees Fahrenheit. Cut the apple into thin slices, and spread them on a greased baking sheet. Sprinkle the apple slices with cinnamon, nutmeg, and brown sugar. Bake the apples in the oven for about 25 minutes, or until the edges are golden brown. Enjoy!
Human: Can I do it on a bonfire?
AI:  Yes, you can definitely roast an apple over a bonfire! Start by spearing the apple onto a roasting stick. Hold the stick a few inches above the fire, and turn the apple every few minutes until it's evenly cooked. O

' Unfortunately, microwaving an apple would not bring out the same flavor as roasting it in an oven or over a bonfire. The texture could also be affected. I would not recommend microwaving an apple.'

In [13]:
message_history.messages

[HumanMessage(content='Hello, how can I roast an apple?', additional_kwargs={}, example=False),
 AIMessage(content=' Hi there! Roasting an apple is a great way to bring out the flavor of the fruit. To roast an apple, preheat your oven to 375 degrees Fahrenheit. Cut the apple into thin slices, and spread them on a greased baking sheet. Sprinkle the apple slices with cinnamon, nutmeg, and brown sugar. Bake the apples in the oven for about 25 minutes, or until the edges are golden brown. Enjoy!', additional_kwargs={}, example=False),
 HumanMessage(content='Can I do it on a bonfire?', additional_kwargs={}, example=False),
 AIMessage(content=" Yes, you can definitely roast an apple over a bonfire! Start by spearing the apple onto a roasting stick. Hold the stick a few inches above the fire, and turn the apple every few minutes until it's evenly cooked. Once the apple looks and smells done, you can take it off the fire and enjoy!", additional_kwargs={}, example=False),
 HumanMessage(content=

## Manually tinkering with the prompt

You can craft your own prompt (through a `PromptTemplate` object) and still take advantage of the chat memory handling by LangChain:

In [14]:
from langchain import LLMChain, PromptTemplate

In [15]:
template = """You are a quirky chatbot having a
conversation with a human, riddled with puns and silly jokes.

{chat_history}
Human: {human_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)

In [16]:
f_message_history = CassandraChatMessageHistory(
    session_id='conversation-funny-a001',
    session=None,
    keyspace=None,
)
f_message_history.clear()

In [17]:
f_memory = ConversationBufferMemory(
    memory_key="chat_history",
    chat_memory=f_message_history,
)

In [18]:
llm_chain = LLMChain(
    llm=llm, 
    prompt=prompt, 
    verbose=True, 
    memory=f_memory,
)

In [19]:
llm_chain.predict(human_input="Tell me about springs")



> Entering new LLMChain chain...
Prompt after formatting:
You are a quirky chatbot having a
conversation with a human, riddled with puns and silly jokes.


Human: Tell me about springs
AI:

> Finished chain.


" Springs are a great time of year! The birds are singing, the flowers are blooming, and it's the perfect season for a good old fashioned bouncing around!"

In [20]:
llm_chain.predict(human_input='Er ... I mean the other type actually.')



> Entering new LLMChain chain...
Prompt after formatting:
You are a quirky chatbot having a
conversation with a human, riddled with puns and silly jokes.

Human: Tell me about springs
AI:  Springs are a great time of year! The birds are singing, the flowers are blooming, and it's the perfect season for a good old fashioned bouncing around!
Human: Er ... I mean the other type actually.
AI:

> Finished chain.


' Oh, you mean the metal kind? Well, you can never go wrong with a good spring! They provide the perfect amount of tension and support for a wide variety of applications.'

## What now?

This demo is hosted [here](https://cassio.org/frameworks/langchain/memory-conversationbuffermemory/) at cassio.org.

Discover the other ways you can integrate 
Cassandra/Astra DB with your ML/GenAI needs,
right **within [your favorite framework](https://cassio.org/frameworks/langchain/about/)**.